In [25]:
import pandas as pd
import numpy as np
import psycopg

In [26]:
df = pd.read_csv('Sample-Superstore.csv', encoding='ISO-8859-1')

In [27]:
df.shape

(9994, 21)

In [28]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [29]:
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])

In [30]:
df['Cost'] = round((df['Sales'] - df['Profit'])/df['Quantity'], 4)


In [31]:
df['Sell Price'] = round(df['Sales']/((1 - df['Discount'])*df['Quantity']), 4)

In [32]:
df['Category'].unique()

array(['Furniture', 'Office Supplies', 'Technology'], dtype=object)

In [33]:
df['Sub-Category'].unique()

array(['Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage',
       'Furnishings', 'Art', 'Phones', 'Binders', 'Appliances', 'Paper',
       'Accessories', 'Envelopes', 'Fasteners', 'Supplies', 'Machines',
       'Copiers'], dtype=object)

Preparing the Products Table

In [34]:
# Generating unique Product IDs based on it's name and price

for category in df['Category'].unique():
    for subcategory in df[df['Category'] == category]['Sub-Category'].unique():
        products_grouped = df[(df['Category'] == category) & (df['Sub-Category'] == subcategory)].groupby(['Product Name', 'Cost'])
        for i, (key, item) in enumerate(products_grouped):
            group = products_grouped.get_group(key)
            cat = group['Category'].unique()[0][:3].upper()
            sub = group['Sub-Category'].unique()[0][:3].upper()
            n = '0'*(8-len(str(i))) + str(i)
            item['id'] = cat[:3] + '-' + sub[:3] + '-' + n
            for index, row in item['id'].items():
                df.loc[index, 'id'] = row


In [35]:
product = pd.DataFrame()
for key, item in df.groupby('id'):
    p = item.iloc[0]
    l = len(product)
    product.loc[l, 'id'] = key
    product.loc[l, 'name'] = p['Product Name']
    product.loc[l, 'cost'] = p['Cost']
    product.loc[l, 'sell_price'] = p['Sell Price']
    product.loc[l, 'category'] = p['Category']
    product.loc[l, 'sub_category'] = p['Sub-Category']

product

,id,name,cost,sell_price,category,sub_category
0,FUR-BOO-00000000,"Atlantic Metals Mobile 2-Shelf Bookcases, Cust...",171.0958,240.98,Furniture,Bookcases
1,FUR-BOO-00000001,"Atlantic Metals Mobile 3-Shelf Bookcases, Cust...",193.1252,260.98,Furniture,Bookcases
2,FUR-BOO-00000002,"Atlantic Metals Mobile 4-Shelf Bookcases, Cust...",196.6860,280.98,Furniture,Bookcases
3,FUR-BOO-00000003,"Atlantic Metals Mobile 5-Shelf Bookcases, Cust...",210.6860,300.98,Furniture,Bookcases
4,FUR-BOO-00000004,Bestar Classic Bookcase,80.9919,99.99,Furniture,Bookcases
...,...,...,...,...,...,...
1886,TEC-PHO-00000184,iKross Bluetooth Portable Keyboard + Cell Phon...,15.0840,20.95,Technology,Phones
1887,TEC-PHO-00000185,iOttie HLCRIO102 Car Mount,18.9905,19.99,Technology,Phones
1888,TEC-PHO-00000186,iOttie XL Car Mount,19.5902,19.99,Technology,Phones
1889,TEC-PHO-00000187,invisibleSHIELD by ZAGG Smudge-Free Screen Pro...,9.3548,17.99,Technology,Phones


Preparing the Customers Table

In [53]:
df[df['id'] == 'OFF-BIN-00000108']['Product Name'].unique()

array(['GBC Premium Transparent Covers with Diagonal Lined Pattern'],
      dtype=object)

In [58]:
df[df['id'] == 'OFF-BIN-00000108'].groupby('Segment')['Quantity'].sum()

Segment
Consumer       35
Corporate      15
Home Office    17
Name: Quantity, dtype: int64

In [36]:
# Deleteting '-' from customer names

df['Customer Name'] = df['Customer Name'].str.rstrip('-')
df['Customer Name'] = df['Customer Name'].str.replace('-', ' ')

In [37]:
df[df['Customer Name'].str.endswith('-')]['Customer Name']

Series([], Name: Customer Name, dtype: object)

In [38]:
df[df['Customer Name'].str.contains(r'-')]['Customer Name']

Series([], Name: Customer Name, dtype: object)

In [39]:
customer = pd.DataFrame()

for key, item in df.groupby('Customer ID'):
    c = item.iloc[0]
    l = len(customer)
    name = c['Customer Name'].split(' ')
    customer.loc[l, 'id'] = key
    customer.loc[l, 'first_name'] = name[0]
    customer.loc[l, 'last_name'] = name[1]
    customer.loc[l, 'segment'] = c['Segment']

customer

,id,first_name,last_name,segment
0,AA-10315,Alex,Avila,Consumer
1,AA-10375,Allen,Armold,Consumer
2,AA-10480,Andrew,Allen,Consumer
3,AA-10645,Anna,Andreadi,Consumer
4,AB-10015,Aaron,Bergman,Consumer
...,...,...,...,...
788,XP-21865,Xylona,Preis,Consumer
789,YC-21895,Yoseph,Carroll,Corporate
790,YS-21880,Yana,Sorensen,Corporate
791,ZC-21910,Zuschuss,Carroll,Consumer


Preparing the Address Table

In [40]:
# Generating unique Address IDs

address_group = df.groupby(['Country', 'Region', 'State', 'City', 'Postal Code'])
for i, (key, item) in enumerate(address_group):
    item['address_id'] = '0'*(8-len(str(i))) + str(i)
    for index, row in item['address_id'].items():
        df.loc[index, 'address_id'] = row

In [41]:
address = pd.DataFrame()

for key, item in df.groupby('address_id'):
    a = item.iloc[0]
    l = len(address)
    address.loc[l, 'id'] = key
    address.loc[l, 'country'] = a['Country']
    address.loc[l, 'region'] = a['Region']
    address.loc[l, 'state'] = a['State']
    address.loc[l, 'city'] = a['City']
    address.loc[l, 'postal_code'] = a['Postal Code']

address

,id,country,region,state,city,postal_code
0,00000000,United States,Central,Illinois,Arlington Heights,60004.0
1,00000001,United States,Central,Illinois,Aurora,60505.0
2,00000002,United States,Central,Illinois,Bloomington,61701.0
3,00000003,United States,Central,Illinois,Bolingbrook,60440.0
4,00000004,United States,Central,Illinois,Buffalo Grove,60089.0
...,...,...,...,...,...,...
627,00000627,United States,West,Washington,Seattle,98105.0
628,00000628,United States,West,Washington,Seattle,98115.0
629,00000629,United States,West,Washington,Spokane,99207.0
630,00000630,United States,West,Washington,Vancouver,98661.0


Preparing the Resident Table

In [42]:
resident = pd.DataFrame()

for key, item in df.groupby(['address_id', 'Customer ID']):
    l = len(resident)
    resident.loc[l, 'address_id'] = key[0]
    resident.loc[l, 'customer_id'] = key[1]

resident

,address_id,customer_id
0,00000000,SC-20845
1,00000001,AH-10120
2,00000001,AW-10930
3,00000001,BE-11410
4,00000001,BF-11020
...,...,...
4905,00000629,MO-17800
4906,00000630,AR-10825
4907,00000630,FM-14290
4908,00000630,JW-15220


Preparing the Shipment Table

In [43]:
# Generating unique Shipment IDs

ship_group = df.groupby(['Order ID'])
for i, (key, item) in enumerate(ship_group):
    group = ship_group.get_group(key[0])
    country_char = group['Country'].unique()[0][:2].upper()
    state_char = group['State'].unique()[0][:2].upper()
    np.random.seed(i)
    random_n = np.random.randint(1, 10000000)
    n = '0'*(8-len(str(random_n))) + str(random_n)
    item['shipment_id'] = country_char + state_char + n
    for index, row in item['shipment_id'].items():
        df.loc[index, 'shipment_id'] = row

In [44]:
shipment = pd.DataFrame()

for key, item in df.groupby('shipment_id'):
    s = item.iloc[0]
    l = len(shipment)
    shipment.loc[l, 'id'] = key
    shipment.loc[l, 'order_id'] = s['Order ID']
    shipment.loc[l, 'ship_mode'] = s['Ship Mode']
    shipment.loc[l, 'address_id'] = s['address_id']
    shipment.loc[l, 'ship_date'] = s['Ship Date']
    
shipment

,id,order_id,ship_mode,address_id,ship_date
0,UNAL00499332,CA-2017-114055,Second Class,00000319,2017-12-29
1,UNAL00923535,CA-2015-102582,Standard Class,00000320,2015-09-19
2,UNAL01309815,CA-2017-127803,Standard Class,00000319,2017-05-23
3,UNAL02012923,CA-2017-102554,Standard Class,00000315,2017-06-15
4,UNAL02250261,CA-2015-154291,Standard Class,00000321,2015-07-29
...,...,...,...,...,...
5004,UNWI08966512,CA-2017-117324,Standard Class,00000188,2017-12-13
5005,UNWI09033334,CA-2016-123540,Second Class,00000189,2016-04-06
5006,UNWI09151816,CA-2016-101448,Second Class,00000187,2016-03-01
5007,UNWI09869447,US-2016-117793,Standard Class,00000190,2016-08-29


Preparing Order Table

In [45]:
order = pd.DataFrame()

for key, item in df.groupby('Order ID'):
    o = item.iloc[0]
    l = len(order)
    order.loc[l, 'id'] = key
    order.loc[l, 'order_date'] = o['Order Date']
    order.loc[l, 'customer_id'] = o['Customer ID']

order

,id,order_date,customer_id
0,CA-2014-100006,2014-09-07,DK-13375
1,CA-2014-100090,2014-07-08,EB-13705
2,CA-2014-100293,2014-03-14,NF-18475
3,CA-2014-100328,2014-01-28,JC-15340
4,CA-2014-100363,2014-04-08,JM-15655
...,...,...,...
5004,US-2017-168802,2017-11-03,JO-15145
5005,US-2017-169320,2017-07-23,LH-16900
5006,US-2017-169488,2017-09-07,AA-10375
5007,US-2017-169502,2017-08-28,MG-17650


Preparing Order Items Table

In [46]:
orderitems = pd.DataFrame()

for key, item in df.groupby(['id', 'Order ID']):
    oi = item.iloc[0]
    l = len(orderitems)
    orderitems.loc[l, 'product_id'] = key[0]
    orderitems.loc[l, 'order_id'] = key[1]
    orderitems.loc[l, 'discount'] = oi['Discount']
    orderitems.loc[l, 'quantity'] = oi['Quantity']
    orderitems.loc[l, 'shipment_id'] = oi['shipment_id']

orderitems

,product_id,order_id,discount,quantity,shipment_id
0,FUR-BOO-00000000,CA-2016-106383,0.70,1.0,UNCO05551802
1,FUR-BOO-00000000,CA-2017-124674,0.32,2.0,UNTE04748006
2,FUR-BOO-00000001,CA-2014-155796,0.50,4.0,UNPE03090177
3,FUR-BOO-00000001,CA-2015-102281,0.20,3.0,UNNE06105748
4,FUR-BOO-00000001,CA-2015-111514,0.15,7.0,UNCA08107212
...,...,...,...,...,...
9981,TEC-PHO-00000188,CA-2015-140144,0.20,3.0,UNCA07576156
9982,TEC-PHO-00000188,CA-2015-156335,0.00,2.0,UNNE06046307
9983,TEC-PHO-00000188,CA-2016-108882,0.20,9.0,UNCA01055175
9984,TEC-PHO-00000188,CA-2017-111647,0.20,4.0,UNTE06635814


In [50]:
with psycopg.connect('dbname=store user=postgres password=postgresql') as conn:

    with conn.cursor() as cur:

        cur.execute("""
            CREATE TABLE IF NOT EXISTS customer (
            "id" varchar PRIMARY KEY,
            "first_name" varchar(128),
            "last_name" varchar(128),
            "segment" varchar(128)
            );

            CREATE TABLE IF NOT EXISTS address (
            "id" integer PRIMARY KEY,
            "country" varchar(128),
            "region" varchar(128),
            "state" varchar(128),
            "city" varchar(128),
            "postal_code" integer
            );

            CREATE TABLE IF NOT EXISTS resident (
            "customer_id" varchar(16),
            "address_id" integer,
            PRIMARY KEY ("customer_id", "address_id"),
            FOREIGN KEY ("customer_id") REFERENCES "customer" ("id"),
            FOREIGN KEY ("address_id") REFERENCES "address" ("id")
            );

            CREATE TABLE IF NOT EXISTS product (
            "id" varchar(16) PRIMARY KEY,
            "name" text,
            "cost" float,
            "sell_price" float,
            "category" varchar(128),
            "sub_category" varchar(128)
            );

            CREATE TABLE  IF NOT EXISTS orders (
            "id" varchar(32) PRIMARY KEY,
            "order_date" date,
            "customer_id" varchar(16),
            FOREIGN KEY ("customer_id") REFERENCES "customer" ("id")
            );

            CREATE TABLE IF NOT EXISTS shipment (
            "id" varchar(64) PRIMARY KEY,
            "order_id" varchar(32),
            "ship_mode" varchar(64),
            "address_id" integer,
            "ship_date" date,
            FOREIGN KEY ("order_id") REFERENCES "orders" ("id"),
            FOREIGN KEY ("address_id") REFERENCES "address" ("id")
            );

            CREATE TABLE IF NOT EXISTS orderitems (
            "product_id" varchar(16),
            "order_id" varchar(32),
            "discount" float,
            "quantity" integer,
            "shipment_id" varchar(64),
            PRIMARY KEY ("product_id", "order_id"),
            FOREIGN KEY ("product_id") REFERENCES "product" ("id"),
            FOREIGN KEY ("order_id") REFERENCES "orders" ("id"),
            FOREIGN KEY ("shipment_id") REFERENCES "shipment" ("id")
            );
        """)

        for item in product.iterrows():
            # Getting the content part of the item, where item[0] is index
            p = item[1]
            cur.execute("INSERT INTO product (id, name, cost, sell_price, category, sub_category) VALUES (%s, %s, %s, %s, %s, %s)", 
                        (p['id'], p['name'], p['cost'], p['sell_price'], p['category'], p['sub_category']))
        
        for item in customer.iterrows():
            c = item[1]
            cur.execute("INSERT INTO customer (id, first_name, last_name, segment) VALUES (%s, %s, %s, %s)", 
                        (c['id'], c['first_name'], c['last_name'], c['segment']))

        for item in address.iterrows():
            a = item[1]
            cur.execute("INSERT INTO address (id, country, region, state, city, postal_code) VALUES (%s, %s, %s, %s, %s, %s)", 
                        (a['id'], a['country'], a['region'], a['state'], a['city'], a['postal_code']))

        for item in resident.iterrows():
            r = item[1]
            cur.execute("INSERT INTO resident (address_id, customer_id) VALUES (%s, %s)", 
                        (r['address_id'], r['customer_id']))

        for item in order.iterrows():
            o = item[1]
            cur.execute('INSERT INTO orders (id, order_date, customer_id) VALUES (%s, %s, %s)', 
                        (o['id'], o['order_date'], o['customer_id']))

        for item in shipment.iterrows():
            s = item[1]
            cur.execute("INSERT INTO shipment (id, order_id, ship_mode, address_id, ship_date) VALUES (%s, %s, %s, %s, %s)", 
                        (s['id'], s['order_id'], s['ship_mode'], s['address_id'], s['ship_date']))

        for item in orderitems.iterrows():
            oi = item[1]
            cur.execute('INSERT INTO orderitems (product_id, order_id, discount, quantity, shipment_id) VALUES (%s, %s, %s, %s, %s)', 
                        (oi['product_id'], oi['order_id'], oi['discount'], oi['quantity'], oi['shipment_id']))